The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed. 

Attribute Information:

Input variables:
# bank client data:
1 - age (numeric)<br>
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')<br>
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)<br>
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')<br>
5 - default: has credit in default? (categorical: 'no','yes','unknown')<br>
6 - housing: has housing loan? (categorical: 'no','yes','unknown')<br>
7 - loan: has personal loan? (categorical: 'no','yes','unknown')<br>
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')<br>
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')<br>
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')<br>
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.<br>
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)<br>
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)<br>
14 - previous: number of contacts performed before this campaign and for this client (numeric)<br>
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')<br>
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)<br>
17 - cons.price.idx: consumer price index - monthly indicator (numeric)<br>
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)<br>
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)<br>
20 - nr.employed: number of employees - quarterly indicator (numeric)<br>

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')<br>

In [1]:
# import important libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline



In [2]:
data = pd.read_csv('bank-additional/bank-additional/bank-additional-full.csv',sep=';')

In [3]:
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
age               41188 non-null int64
job               41188 non-null object
marital           41188 non-null object
education         41188 non-null object
default           41188 non-null object
housing           41188 non-null object
loan              41188 non-null object
contact           41188 non-null object
month             41188 non-null object
day_of_week       41188 non-null object
duration          41188 non-null int64
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
poutcome          41188 non-null object
emp.var.rate      41188 non-null float64
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
nr.employed       41188 non-null float64
y                 41188 non-null object
dtypes: float64(5), int64(5), object(11)
memory usa

In [5]:
data.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [14]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 20 columns):
age               41188 non-null int64
job               41188 non-null object
marital           41188 non-null object
education         41188 non-null object
default           41188 non-null object
housing           41188 non-null object
loan              41188 non-null object
contact           41188 non-null object
month             41188 non-null object
day_of_week       41188 non-null object
duration          41188 non-null int64
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
poutcome          41188 non-null object
emp.var.rate      41188 non-null float64
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
nr.employed       41188 non-null float64
dtypes: float64(5), int64(5), object(10)
memory usage: 6.3+ MB


In [8]:
#data.head()

X = data.drop('y',axis=1)
y = data.y
    
    
    

In [15]:
# before using One hot Encoder we have to convert the categories into Numerical data
# we will use labelEncoder for that. 

from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()

X['job'] = labelEncoder.fit_transform(X['job'])
X['marital'] = labelEncoder.fit_transform(X['marital'])
X['education'] = labelEncoder.fit_transform(X['education'])
X['default'] = labelEncoder.fit_transform(X['default'])
X['housing'] = labelEncoder.fit_transform(X['housing'])
X['loan'] = labelEncoder.fit_transform(X['loan'])
X['contact'] = labelEncoder.fit_transform(X['contact'])
X['month'] = labelEncoder.fit_transform(X['month'])
X['day_of_week'] = labelEncoder.fit_transform(X['day_of_week'])
X['poutcome'] = labelEncoder.fit_transform(X['poutcome'])




In [20]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')

enc = encoder.fit_transform(X)

print(enc)





  (0, 39)	1.0
  (0, 81)	1.0
  (0, 91)	1.0
  (0, 94)	1.0
  (0, 102)	1.0
  (0, 105)	1.0
  (0, 108)	1.0
  (0, 112)	1.0
  (0, 119)	1.0
  (0, 124)	1.0
  (0, 389)	1.0
  (0, 1672)	1.0
  (0, 1740)	1.0
  (0, 1741)	1.0
  (0, 1750)	1.0
  (0, 1760)	1.0
  (0, 1780)	1.0
  (0, 1804)	1.0
  (0, 2101)	1.0
  (0, 2138)	1.0
  (1, 40)	1.0
  (1, 85)	1.0
  (1, 91)	1.0
  (1, 97)	1.0
  (1, 103)	1.0
  :	:
  (41186, 1757)	1.0
  (41186, 1787)	1.0
  (41186, 1788)	1.0
  (41186, 1985)	1.0
  (41186, 2130)	1.0
  (41187, 57)	1.0
  (41187, 83)	1.0
  (41187, 91)	1.0
  (41187, 99)	1.0
  (41187, 102)	1.0
  (41187, 107)	1.0
  (41187, 108)	1.0
  (41187, 111)	1.0
  (41187, 120)	1.0
  (41187, 123)	1.0
  (41187, 367)	1.0
  (41187, 1674)	1.0
  (41187, 1740)	1.0
  (41187, 1742)	1.0
  (41187, 1749)	1.0
  (41187, 1757)	1.0
  (41187, 1787)	1.0
  (41187, 1788)	1.0
  (41187, 1985)	1.0
  (41187, 2130)	1.0


In [ ]:
data_encoded.columns


In [ ]:
data_encoded.info()

In [ ]:
encoder.get_feature_names

In [ ]:
data['loan'].value_counts()

In [ ]:
data.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
sns.pairplot(data, kind='scatter', hue='y')